# Data Acquisition

In [20]:
import pandas as pd
import numpy as np
import wrds
import requests
from tqdm import tqdm

In [ ]:
#connect to WRDS
db = wrds.Connection(wrds_username = 'nmonnet')
db.create_pgpass_file()

## Get a list of all usable stock names

In [3]:
#import table about all available data series on wrds
stocknames = db.get_table(library='crsp', table = 'stocknames')
#change dates to datetime objects
stocknames.end_date = pd.to_datetime(stocknames.end_date)
stocknames.st_date = pd.to_datetime(stocknames.st_date)
#keep only their most recent ticker and Name
stocknames = stocknames[stocknames.nameenddt == stocknames.end_date]
stocknames = stocknames[['ticker','comnam','permco','st_date','end_date']]
#get rid of duplicate ticker and permco stocks
stocknames = stocknames[stocknames.ticker.notnull()]
stocknames.sort_values('end_date', ascending = False, inplace = True)
to_drop = []
#keep the observation that has the most recent observation
#if there are several, keep one one with the oldest observation
duplicated_tickers = stocknames[stocknames.duplicated(subset = 'ticker')].ticker.unique()
for ticker in duplicated_tickers:
    sn = stocknames[stocknames.ticker == ticker]
    sn = sn[sn.end_date == sn.end_date.max()]
    #if several have recent observations keep the one with the furthest start date
    if len(sn) > 1:
        sn = sn[sn.st_date == sn.st_date.min()]
    to_keep = sn.index.values[0]
    indexes = stocknames[stocknames.ticker == ticker].index.values
    to_drop.append(indexes[~np.isin(indexes, to_keep)])
duplicated_permcos = stocknames[stocknames.duplicated(subset = 'permco')].permco.values
for permco in duplicated_permcos:
    sn = stocknames[stocknames.permco == permco]
    sn = sn[sn.end_date == sn.end_date.max()]
    if len(sn) > 1:
        sn = sn[sn.st_date == sn.st_date.min()]
    to_keep = sn.index.values[0]
    indexes = stocknames[stocknames.permco == permco].index.values
    to_drop.append(indexes[~np.isin(indexes, to_keep)])

to_drop = np.concatenate(to_drop).ravel()
stocknames = stocknames.drop(labels = to_drop)
stocknames.reset_index(inplace = True, drop = True)

print("Number of available time series: ", len(stocknames))
display(stocknames)

Number of available time series:  23815


,ticker,comnam,permco,st_date,end_date
0,TSLA,TESLA INC,53453,2010-06-29,2023-12-29
1,CEFS,EXCHANGE LISTED FUNDS TRUST,54446,2017-03-21,2023-12-29
2,HBIO,HARVARD BIOSCIENCE INC,41060,2000-12-07,2023-12-29
3,BELFA,BEL FUSE INC,6653,1983-12-06,2023-12-29
4,AMPX,AMPRIUS TECHNOLOGIES INC,59244,2022-09-15,2023-12-29
...,...,...,...,...,...
23810,ACJ,AMERICAN CHICLE CO,22181,1925-12-31,1962-10-02
23811,KAW,KAWNEER CO,23917,1962-07-02,1962-08-31
23812,BCA,BANCO DE LOS ANDES COLUMBIA,23628,1962-07-02,1962-08-24
23813,NJC,CENTRAL RR CO N J,22245,1925-12-31,1962-07-27


In [4]:
#read in the txt file linking tickers to CIKs #use : https://www.sec.gov/include/ticker.txt
CIK = pd.read_csv('data/ticker_to_CIK.txt', delimiter = "\t", header = None, names = ['ticker', 'CIK'], na_filter = False)
CIK.ticker = CIK.ticker.str.upper()

In [5]:
#stocknames = stocknames.set_index('ticker')
stocknames = stocknames.merge(CIK, on = 'ticker', how = 'left')
#save stocknames as a gzipped csv
stocknames.dropna(inplace=True)
stocknames["CIK"]=stocknames["CIK"].apply(lambda x : int(x))
stocknames.to_csv("data/stocknames.csv.gz", index = False)


display(stocknames)

,ticker,comnam,permco,st_date,end_date,CIK
0,TSLA,TESLA INC,53453,2010-06-29,2023-12-29,1318605
2,HBIO,HARVARD BIOSCIENCE INC,41060,2000-12-07,2023-12-29,1123494
3,BELFA,BEL FUSE INC,6653,1983-12-06,2023-12-29,729580
4,AMPX,AMPRIUS TECHNOLOGIES INC,59244,2022-09-15,2023-12-29,1899287
5,NUW,NUVEEN AMT FREE MUN VALUE FD,53122,2009-02-25,2023-12-29,1450445
...,...,...,...,...,...,...
23494,CLCN,CLARCAN PETROLEUM CORP,23248,1973-02-09,1974-04-01,1394638
23512,BRVO,BRAVO KNITS INC,22896,1972-12-14,1973-12-12,1444839
23572,ATDS,ATLANTIC DEPARTMENT STORES INC,22619,1972-12-14,1973-03-02,1068689
23585,JJN,NEWBERRY J J CO,22928,1934-04-02,1972-09-01,312070


## Get all urls

In [73]:
def get_urls(cik,ticker):
    cik_str=str(cik)
    cik_str_0=cik_str.zfill(10)
    url = 'https://data.sec.gov/submissions/CIK'+cik_str_0+'.json'
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    # Check if the request was successful
    if response.status_code == 200:
        # Get the JSON content from the response
        data = response.json()
    
        # Convert JSON data to pandas DataFrame
        # Assuming the JSON structure has a key 'filings' which contains the relevant data
        # Adjust the key according to the actual structure of the JSON data
    
        # Print the structure of the JSON data to understand its format
        #print(data.keys())  # This will help in understanding what keys are present
    
        # Example for extracting 'filings' part if it exists
        if 'filings' in data:
            filings = data['filings']
    
            # Check the structure of 'filings'
            #print(filings.keys())  # This will help in understanding what sub-keys are present
    
            # Convert 'filings' to a DataFrame if it contains tabular data
            # Here assuming 'filings' is a dictionary with a key 'recent' which is a dictionary
            # containing the tabular data as lists under 'accessionNumber', 'reportDate', etc.
            if 'recent' in filings:
                recent_filings = filings['recent']
                df = pd.DataFrame(recent_filings)
    
                #print("DataFrame created successfully.")
                #print(df.head())  # Display the first few rows of the DataFrame
            else:
                print("'recent' key not found in 'filings'.")
        else:
            print("'filings' key not found in JSON data.")
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")


    df=df[df["form"]=="10-K"]
    df["accessionNumber"]=df["accessionNumber"].apply(lambda x : x.replace("-",""))
    df["url"]="https://www.sec.gov/Archives/edgar/data/"+cik_str+"/"+df["accessionNumber"]+"/"+df["primaryDocument"]
    df["cik"]=cik
    df["ticker"]=ticker
    df["date"]=df["filingDate"].apply(lambda x : x[:4])
    
    #cik#/#accessionNumber/#primaryDocument#"
    if df.shape[0]==0:
        return "ERROR"

    df=df[df['url'].str.contains('\\.htm|\\.txt', regex=True)]
    return df[["date","filingDate","url","cik","ticker"]]

In [74]:
df = pd.DataFrame(columns=["date","filingDate","url","cik","ticker"])
nb_error=0

for i in tqdm(range(stocknames.shape[0])):

    cik=stocknames.iloc[i]["CIK"]
    ticker=stocknames.iloc[i]["ticker"]


    df_to_append=get_urls(cik,ticker)
    if isinstance(df_to_append, str):
        "skip"
        if df_to_append=="ERROR":
            nb_error+=1
        else:
            print("ouch!")
        #print("skip")

    else:
        df=pd.concat([df, df_to_append])
  
    
print(nb_error)   

100%|██████████████████████████████████████████████████████████████████████████████| 7001/7001 [46:53<00:00,  2.49it/s]

1507


In [75]:
display(df)

,date,filingDate,url,cik,ticker
79,2024,2024-01-29,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
180,2023,2023-01-31,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
280,2022,2022-02-07,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
412,2021,2021-02-08,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
550,2020,2020-02-13,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
...,...,...,...,...,...
35,2023,2023-02-24,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
57,2022,2022-03-31,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
73,2021,2021-03-23,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
107,2020,2020-04-17,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS


In [76]:
df.to_csv('data/url_10_Ks.csv.gz', compression='gzip', index=False)

In [ ]:
for i in df["url"]:
    print(i)